**<Shelter Animal Outcomes - Help improve outcomes for shelter animals>**

The data comes from Austin Animal Center from October 1st, 2013 to March, 2016. Outcomes represent the status of animals as they leave the Animal Center. All animals receive a unique Animal ID during intake. 
In this competition, you are going to predict the outcome of the animal as they leave the Animal Center. These outcomes include: Adoption, Died, Euthanasia, Return to owner, and Transfer.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train = pd.read_csv('/kaggle/input/shelter-animal-outcomes/train.csv.gz')
train

In [3]:
test = pd.read_csv('/kaggle/input/shelter-animal-outcomes/test.csv.gz')
test

In [4]:
display(train,test)

In [5]:
alldata = pd.concat([train,test])
alldata

In [6]:
alldata['DateTime'] = pd.to_datetime(alldata['DateTime'])

In [7]:
alldata['year'] = alldata['DateTime'].dt.year  # year이 도움이 되는 이유 추측해보면 숫자 느낌으로는 시간이 가면서 사람들의 인식 변화, 카테고리 느낌으로는 특정 연도에 규정 변화 등의 큰 사건 발생 가능성 등
alldata['month'] = alldata['DateTime'].dt.month  # month의 경우 계절, 연말 연초 등 영향
alldata['day'] = alldata['DateTime'].dt.day
alldata['hour'] = alldata['DateTime'].dt.hour
alldata['weekday'] = alldata['DateTime'].dt.weekday  # 주중인지 주말인지 영향 받음
alldata['week'] = alldata['DateTime'].dt.week
alldata['min'] = alldata['DateTime'].dt.minute  # 분 정보도 도움이 되는 경우가 많음
alldata['time'] = alldata['DateTime'].dt.date-alldata['DateTime'].dt.date.min()  # 연도는 너무 범위가 넓음. 시간의 흐름에 따른 변화를 고려해줄 수 있도록. 해당 날짜에서 제일 빠른 날짜를 빼면 시간이 얼마나 지났는지 알 수 있음.

In [8]:
alldata['time'] = alldata['time'].apply(lambda x:x.days)  # time열 각각의 데이터에 접근해서 적용. 날짜 형식이었는데 days 빼고 숫자만 남기기.

In [9]:
alldata['AgeuponOutcome'].unique()

In [10]:
def age(x):
    if pd.isnull(x):
        return -1  # null값 먼저 채워줌
    num = int(x.split()[0])  # 2 years 이런거에서 앞에 숫자만 가져오기 (0열만 가져오기)
    if 'year' in x:
        return num*365
    elif 'month' in x:
        return num*30
    elif 'week' in x:
        return num*7
    else :
        return num

In [11]:
alldata['AgeuponOutcome'] = alldata['AgeuponOutcome'].apply(age)
alldata

In [12]:
alldata2 = alldata.drop(columns=['AnimalID','DateTime','OutcomeType','OutcomeSubtype','ID'])
alldata2

In [13]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

c = alldata2.columns[alldata2.dtypes == object]

for i in c:
    alldata2[i] = le.fit_transform(alldata2[i])
    
alldata2

In [14]:
train2 = alldata2[:len(train)]
test2 = alldata2[len(train):]

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# rf = RandomForestClassifier(n_jobs=4,random_state=42)
# rf.fit(train2,train['OutcomeType'])
# result = rf.predict_proba(test2)
# result

In [15]:
from catboost import CatBoostClassifier
cbc = CatBoostClassifier()
cbc.fit(train2,train['OutcomeType'])
result = cbc.predict_proba(test2)

In [16]:
sub = pd.read_csv('/kaggle/input/shelter-animal-outcomes/sample_submission.csv.gz')
sub.iloc[:,1:] = result  # iloc: 데이터를 어느 위치에 넣을건지 (행은 전체, 열은 1열부터)
sub

In [ ]:
sub.to_csv('sub.csv',index=0)